In [83]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers 
import bert 
import re 
# re — Regular expression operations
import math
import csv
import pandas as pd                     
import cv2
from PIL import Image
from tensorflow.keras import losses
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
data = pd.read_csv("/Users/zimingfang/Desktop/Animated GIFs/AwesomeGif/tgif-v1.0.tsv", sep='\t')
data.isnull().values.any()
data.shape

(125782, 2)

In [3]:
# data.head()

In [4]:
gif_links = list(data.y.values)
raw_tweets = list(data.x.values)

In [5]:
# gif_links

In [6]:
# raw_tweets

# Tweet Pre-Process 

## Remove Special Char

In [7]:
# definition for function for removing html tags 
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [8]:
# definition for function for remove any punctuations and special characters
def preprocess_text(raw_tweaet):
    # Removing html tags
    tweet = remove_tags(raw_tweaet)
    # Removing html tags
    tweet = re.sub('[^a-zA-Z]', '', tweet)
    # Removing html tags
    tweet = re.sub(r"\s+[a-zA-Z]\s+", ' ', tweet)
    # Removing multiple spaces
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

In [9]:
# run the preprocess_text function to clean tweets list 
tweets = [] 
for tweet in raw_tweets[:50]:
    tweets.append(preprocess_text(tweet))

## Tokenizing 

In [10]:
# Create a tokenizer 
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# .numpy(): converts a tensor object into an numpy.ndarray
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [11]:
# Definition for function for convert tweet to ids 
def tokenize_tweets(text_tweets):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_tweets))

In [12]:
# run the tokenize_tweets on tweets 
tokenized_tweets = [tokenize_tweets(tweet) for tweet in tweets]

# GIF Pre-Process

In [13]:
import requests
# Requests is an elegant and simple HTTP library for Python
import os 
# os — Miscellaneous operating system interfaces¶
os.chdir('/Users/zimingfang/Desktop/Animated GIFs/AwesomeGif/gifs')
# !pwd

In [14]:
def gif_downloader(image_urls, status=[], filenames = []):
    
    for index, img in enumerate(image_urls):
        # We can split the file based upon / and extract the last split within the python list below:
        file_name = img.split('/')[-1]
        #print("fThis is the file name: {file_name}")
        filenames.append(file_name) 
        # Now let's send a request to the image URL:
        r = requests.get(img, stream=True)
        # We can check that the status code is 200 before doing anything else:
        if r.status_code == 200:
            # This command below will allow us to write the data to a file as binary:
            with open(file_name, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
            status.append(True)
        else:
            # We will write all of the images back to the broken_images list:
            status.append(False)
    return filenames, status

In [15]:
# def load_gif(image_path, sess):
#     image = tf.io.read_file(image_file)
#     image = tf.io.decode_gif(image)
#     return sess.run(image)

In [16]:
download_status = []
filenames = []
filenames, download_status = gif_downloader(gif_links[:50], status=download_status)
# if any gif was not downloaded successfully 
print(any(status == False for status in download_status))

False


In [17]:
# filenames

In [18]:
def get_avg_fps(PIL_Image_object):
    """ Returns the average framerate of a PIL Image object """
    PIL_Image_object.seek(0)
    frames = duration = 0
    while True:
        try:
            frames += 1
            duration += PIL_Image_object.info['duration']
            PIL_Image_object.seek(PIL_Image_object.tell() + 1)
        except EOFError:
            return frames / duration * 1000
    return None

In [19]:
# In GIF files, each frame has its own duration. So there is no general fps for a GIF file. 
framerates = []
for gif_file_path in filenames: 
#     cap=cv2.VideoCapture("gif_file_path")
#     fps = cap.get(cv2.CAP_PROP_FPS)
    gif_obj = Image.open(gif_file_path)
    framerates.append(get_avg_fps(gif_obj))

In [20]:
# framerates

# Prerparing Data For Training


In [21]:
x_train = tokenized_tweets
y_train = framerates

In [26]:
# len(x_train)

In [25]:
tweets_with_len = [[tweet, y_train[i], len(tweet)]
                 for i, tweet in enumerate(x_train)]

In [27]:
#sort the data by tweet length 
tweets_with_len.sort(key=lambda x: x[2])
#remove the tweet length attribute from dataset 
sorted_tweets_labels = [(tweet_lab[0], tweet_lab[1]) for tweet_lab in tweets_with_len]

In [29]:
# sorted_tweets_labels

In [33]:
# convert the sorted dataset into a TensorFlow 2.0-compliant input dataset shape
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_tweets_labels, output_types=(tf.int32, tf.float32))


In [34]:
BATCH_SIZE = 10 
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [82]:
# next(iter(batched_dataset))

# Creating the Model


In [72]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200

In [88]:
#GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension.
#allows the model to handle input of variable length, in the simplest way possible

model = tf.keras.Sequential([
    layers.Embedding(VOCAB_LENGTH + 1, EMB_DIM),
    layers.Dropout(0.2),    
    layers.GlobalAveragePooling1D(), 
    layers.Dropout(0.2),
    layers.Dense(1, activation='linear') #set activation to linear for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 200)         6104600   
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 200)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 201       
Total params: 6,104,801
Trainable params: 6,104,801
Non-trainable params: 0
_________________________________________________________________


In [89]:
epochs = 100
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs)

Epoch 1/100
4/4 [==============================] - 1s 105ms/step - loss: 192.5695 - mae: 12.2242 - val_loss: 155.0065 - val_mae: 11.5399
Epoch 2/100
4/4 [==============================] - 0s 60ms/step - loss: 191.6136 - mae: 12.1848 - val_loss: 154.4618 - val_mae: 11.5159
Epoch 3/100
4/4 [==============================] - 0s 57ms/step - loss: 190.5990 - mae: 12.1409 - val_loss: 153.8709 - val_mae: 11.4898
Epoch 4/100
4/4 [==============================] - 0s 57ms/step - loss: 189.6089 - mae: 12.1003 - val_loss: 153.2388 - val_mae: 11.4619
Epoch 5/100
4/4 [==============================] - 0s 69ms/step - loss: 188.4347 - mae: 12.0510 - val_loss: 152.5599 - val_mae: 11.4319
Epoch 6/100
4/4 [==============================] - 0s 56ms/step - loss: 187.3515 - mae: 12.0034 - val_loss: 151.8275 - val_mae: 11.3995
Epoch 7/100
4/4 [==============================] - 0s 56ms/step - loss: 186.1600 - mae: 11.9534 - val_loss: 151.0382 - val_mae: 11.3644
Epoch 8/100
4/4 [==============================

Epoch 62/100
4/4 [==============================] - 0s 51ms/step - loss: 43.5951 - mae: 3.2958 - val_loss: 58.9398 - val_mae: 6.5489
Epoch 63/100
4/4 [==============================] - 0s 50ms/step - loss: 41.1602 - mae: 3.1245 - val_loss: 58.1426 - val_mae: 6.4980
Epoch 64/100
4/4 [==============================] - 0s 51ms/step - loss: 40.5123 - mae: 3.2756 - val_loss: 57.3867 - val_mae: 6.4490
Epoch 65/100
4/4 [==============================] - 0s 52ms/step - loss: 40.1970 - mae: 3.1369 - val_loss: 56.6876 - val_mae: 6.4035
Epoch 66/100
4/4 [==============================] - 0s 53ms/step - loss: 37.3339 - mae: 2.9653 - val_loss: 56.0246 - val_mae: 6.3596
Epoch 67/100
4/4 [==============================] - 0s 52ms/step - loss: 36.7139 - mae: 2.9767 - val_loss: 55.4098 - val_mae: 6.3182
Epoch 68/100
4/4 [==============================] - 0s 53ms/step - loss: 36.9138 - mae: 2.9842 - val_loss: 54.8325 - val_mae: 6.2787
Epoch 69/100
4/4 [==============================] - 0s 54ms/step - lo

In [86]:
print(history.history.keys())

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


In [87]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                     name='Valid'))

fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [79]:
results = model.evaluate(test_data)

print(results)

1/1 [==============================] - 0s 43ms/step - loss: 48.5708 - mae: 5.8409
[48.570796966552734, 5.840874671936035]
